## Install

In [1]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 37.9 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import fitz  # PyMuPDF
import re
import logging

## 990 Before 2018

In [ ]:
def setup_logging():
    """
    Setup logging configuration.
    """
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.
    """
    text = ""
    try:
        # Open the PDF
        pdf_document = fitz.open(pdf_path)

        # Iterate through each page
        for page_number in range(len(pdf_document)):
            page = pdf_document.load_page(page_number)

            # Extract text from the page
            text += page.get_text()

        # Close the PDF
        pdf_document.close()
    except Exception as e:
        logging.error(f"Error occurred while extracting text from {pdf_path}: {e}")

    return text

def extract_info(pdf_text, patterns):
    """
    Extract information from PDF text based on provided patterns.
    """
    info_dict = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, pdf_text, re.DOTALL)
        if match:
            info_dict[key] = match.group(1).strip()
    return info_dict

def process_pdf(pdf_path, patterns):
    """
    Process a single PDF file.
    """
    # Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_path)

    # Extract information from PDF text
    info_dict = extract_info(pdf_text, patterns)

    # Initialize DataFrame
    df = pd.DataFrame(info_dict, index=[0])

    return df

def main(pdf_directory, patterns):
    """
    Main function to extract information from multiple PDFs and concatenate results.
    """
    # List PDF files in the directory
    pdf_files = [file for file in os.listdir(pdf_directory) if file.endswith('.pdf')]

    # Process each PDF file
    dfs = []
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        df = process_pdf(pdf_path, patterns)
        dfs.append(df)

    # Concatenate DataFrames
    final_df = pd.concat(dfs, ignore_index=True)

    return final_df

if __name__ == "__main__":
    setup_logging()
    pdf_directory = 'Directory Including 990-EZ Before 2019'  # Replace
    patterns = {
        "Organization": r"Address change \n(.*?)\nD Name change",
        "Year": r"Exempt From Income Tax \n(.*?)\nForm990-EZ",
        "Street": r"Initial return \n(.*?)\nD Final return",
        "CityStateZip": r"and ZIP or foreign postal code \n(.*?)\nF Group Exemption",
        "Receipts": r"• • • • ~ (.*?)\nRevenue, Expenses,",
        "Revenue": r"Add lines 1, 2, 3, 4, 5c, 6d, 7c, and 8 \n• \n9 \n(.*?)\n10"
    }
    try:
        df = main(pdf_directory, patterns)

    except Exception as e:
        logging.error(f"An error occurred: {e}")
